In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
os.getenv('GROQ_API_KEY')

'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'

In [4]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key=os.getenv('GROQ_API_KEY'), 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [ ]:
from langchain_community.document_loaders  import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-44289")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer II, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. doe

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
job = json_res[0]

In [7]:
type(json_res)

dict

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

C:\Users\Akash Deep Patel\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:46<00:00, 785kiB/s]   


In [13]:
job['skills']

['front-end frameworks like React, Angular, or Vue.js',
 'cloud architecture, modern DevOps, infrastructure as code, CI/CD and related tools',
 'AWS products including Lambda, Step Functions, DynamoDB, Elasticsearch, s3',
 'Modern testing methodologies and frameworks such as Mocha, Jasmine and Jest',
 'architectural design patterns and computer-science fundamentals',
 'implementing and integrating AI, Machine Learning and related data solutions',
 'modern software architectural principles and patterns (REST, domain-driven design, microservices, etc)']

In [12]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}]]

In [15]:
job

{'role': 'Software Engineer II, AI/ML Platforms',
 'experience': '2+ years professional software development experience',
 'skills': ['Strong Computer Science fundamentals',
  'Programming experience with at least one modern language such as Java, Python, Golang',
  'Experience with Infrastructure as Code / DevOps practices and technologies like Terraform',
  'Experience with Cloud services (e.g. AWS Cloud EC2, S3, DynamoDB, Azure Cloud, etc.)',
  'Experience developing and using microservices'],
 'description': 'We are looking for a Software Engineer II to design and implement core components of our data science services and platform and scale it to serve Nike globally. The focus of this engineer will be at the intersection of computer systems and machine learning. You will also work collaboratively with the data science and analytics teams to ship scalable, performant and reliable solutions.'}

In [15]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [16]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's ITC Team

Dear Hiring Manager,

I came across the job posting for a Software Engineer II, ITC at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your goals. With our expertise in software development, we can support Nike's pursuit of delivering state-of-the-art tools to the Consumer Product & Innovation community.

Our team at AtliQ has extensive experience in front-end frameworks like React, Angular, and Vue.js, which aligns perfectly with the job requirements. We've worked on numerous projects involving cloud architecture, modern DevOps, infrastructure as code, CI/CD, and related tools, including AWS products like Lambda, Step Functions, DynamoDB, Elasticsearch, and S3.

Our portfolio showcases our capabilities in:

* Front-end development: https://example.com/vue-portfolio
* DevOps: https://example.com/devops-portfolio
* Full-stack JavaScript development: https://example